# D09 Biến đổi dữ liệu với `groupby()`

## Mục đích

Tổng hợp các tính năng kết hợp cùng với `groupby()` để biến đổi dữ liệu.

## Ví dụ 1: Chuẩn hóa số liệu

Thông thường khi phân tích số liệu, bạn sẽ giữ nguyên các giá trị của các trường. Tuy nhiên, trong một số kịch bản nhất định, chúng ta phải chuẩn hóa (normalize) số liệu trước khi phân tích. Cách chuẩn hóa thường gặp nhất là sử dụng công thức sau:

$$ x_{\text{chuẩn hóa}} = \frac{x - m}{s} $$

Trong đó, $x$ là giá trị gốc cần được chuẩn hóa, $m$ là trung bình của tất cả các giá trị, và $s$ là độ lệch chuẩn của tất cả các giá trị này.

Nếu chỉ thực hiện công việc này trên toàn bộ số liệu, bạn không cần kĩ thuật gì phức tạp.

In [1]:
import pandas as pd

data = pd.Series(range(5))
data

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [2]:
(data - data.mean()) / data.std()

0   -1.264911
1   -0.632456
2    0.000000
3    0.632456
4    1.264911
dtype: float64

Nhưng nếu bạn cần chuẩn hóa theo nhóm (ví dụ, chuẩn hóa các giá trị của nam riêng, của nữ riêng), bạn sẽ cần đến các công cụ kết hợp với `groupby()`. Ví dụ như dữ liệu dưới đây (nếu bạn để ý thì kết quả chuẩn hóa của nam và nữ sẽ giống nhau).

In [3]:
d = pd.DataFrame({
    "sex": ["Nam"] * 5 + ["Nữ"] * 5,
    "value": range(10)
})
d

,sex,value
0,Nam,0
1,Nam,1
2,Nam,2
3,Nam,3
4,Nam,4
5,Nữ,5
6,Nữ,6
7,Nữ,7
8,Nữ,8
9,Nữ,9


Công cụ mà chúng ta sẽ sử dụng là hàm `transform()`. Trong khi hàm `agg()` nhận các giá trị của data frame ở trong từng nhóm và trả về một giá trị "tổng hợp", hàm `transform()` nhận các giá trị của data frame, biến đổi chúng, và trả về đúng như cấu trúc ban đầu.

In [4]:
d1 = d.copy(deep=True)
d1["value_normalized"] = d1.groupby("sex").transform(lambda x: (x - x.mean()) / x.std())
d1

,sex,value,value_normalized
0,Nam,0,-1.264911
1,Nam,1,-0.632456
2,Nam,2,0.000000
3,Nam,3,0.632456
4,Nam,4,1.264911
5,Nữ,5,-1.264911
6,Nữ,6,-0.632456
7,Nữ,7,0.000000
8,Nữ,8,0.632456
9,Nữ,9,1.264911


## Ví dụ 2: Tính tỉ lệ phần trăm theo nhóm

Tương tự như ví dụ trên, việc tính tỉ lệ phần trăm cho toàn bộ dữ liệu thì đơn giản.

In [5]:
d2 = d.copy(deep=True)
d2["pct1"] = d2["value"] / d2["value"].sum() * 100
d2

,sex,value,pct1
0,Nam,0,0.000000
1,Nam,1,2.222222
2,Nam,2,4.444444
3,Nam,3,6.666667
4,Nam,4,8.888889
5,Nữ,5,11.111111
6,Nữ,6,13.333333
7,Nữ,7,15.555556
8,Nữ,8,17.777778
9,Nữ,9,20.000000


Và để tính riêng theo nhóm, chúng ta sẽ dùng hàm `transform()`.

In [6]:
d2["pct2"] = d2.groupby("sex")["value"].transform(lambda x: x / x.sum() * 100)
d2

,sex,value,pct1,pct2
0,Nam,0,0.000000,0.000000
1,Nam,1,2.222222,10.000000
2,Nam,2,4.444444,20.000000
3,Nam,3,6.666667,30.000000
4,Nam,4,8.888889,40.000000
5,Nữ,5,11.111111,14.285714
6,Nữ,6,13.333333,17.142857
7,Nữ,7,15.555556,20.000000
8,Nữ,8,17.777778,22.857143
9,Nữ,9,20.000000,25.714286


Trong một số trường hợp, bạn cần tạo một cột mới chứa số liệu tổng hợp (ví dụ, tổng, trung bình, v.v.) theo từng nhóm. Bạn cũng có thể dùng hàm `transform()` cho việc này.

In [7]:
d2["value_total"] = d2.groupby("sex")["value"].transform("sum")
d2

,sex,value,pct1,pct2,value_total
0,Nam,0,0.000000,0.000000,10
1,Nam,1,2.222222,10.000000,10
2,Nam,2,4.444444,20.000000,10
3,Nam,3,6.666667,30.000000,10
4,Nam,4,8.888889,40.000000,10
5,Nữ,5,11.111111,14.285714,35
6,Nữ,6,13.333333,17.142857,35
7,Nữ,7,15.555556,20.000000,35
8,Nữ,8,17.777778,22.857143,35
9,Nữ,9,20.000000,25.714286,35


## Ví dụ 3: Tạo điều kiện theo nhóm

Hàm `transform()` cũng rất thuận tiện khi bạn cần tạo điều kiện để lọc một số hàng trong bộ số liệu, và điều kiện này lại là điều kiện dựa trên cả nhóm. Ví dụ, bạn muốn lọc ra các hàng của các nhóm có từ 3 bản ghi có giá trị >2.

Đầu tiên chúng ta hãy quan sát từng bước lập điều kiện. Điều kiện thứ nhất là "giá trị >2".

In [8]:
d3 = d.copy(deep=True)
d3["value_gt2"] = d3["value"].gt(2)
d3

,sex,value,value_gt2
0,Nam,0,False
1,Nam,1,False
2,Nam,2,False
3,Nam,3,True
4,Nam,4,True
5,Nữ,5,True
6,Nữ,6,True
7,Nữ,7,True
8,Nữ,8,True
9,Nữ,9,True


Bằng mắt thường, bạn cũng có thể đếm được nhóm nam có 2 bản ghi và nữ có 5 bản ghi thỏa mãn điều kiện. Chúng ta có thể dùng `groupby()` để đếm.

In [9]:
d3.groupby("sex")["value_gt2"].sum()

sex
Nam    2
Nữ     5
Name: value_gt2, dtype: int64

Vậy nếu chỉ lọc ra các hàng của nhóm có từ 3 bản ghi thỏa mãn điều kiện này, chúng ta sẽ lọc ra toàn bộ các hàng của nhóm nữ.

In [10]:
d3.groupby("sex")["value_gt2"].sum().gt(3)

sex
Nam    False
Nữ      True
Name: value_gt2, dtype: bool

Hàm `transform()` sẽ giúp chúng ta chuyển điều kiện này vào từng hàng trong data frame gốc.

In [11]:
d3["chk_value_gt2"] = d3.groupby("sex")["value"].transform(lambda x: x.gt(2).sum() > 3)
d3

,sex,value,value_gt2,chk_value_gt2
0,Nam,0,False,False
1,Nam,1,False,False
2,Nam,2,False,False
3,Nam,3,True,False
4,Nam,4,True,False
5,Nữ,5,True,True
6,Nữ,6,True,True
7,Nữ,7,True,True
8,Nữ,8,True,True
9,Nữ,9,True,True


Nếu bạn không cần lưu lại các giá trị này trong data frame mà chỉ dùng chúng làm điều kiện lọc, bạn có thể lưu chúng trong một biến khác.

In [12]:
chk_value_gt2 = d3.groupby("sex")["value"].transform(lambda x: x.gt(2).sum() > 3)
d3.loc[chk_value_gt2]

,sex,value,value_gt2,chk_value_gt2
5,Nữ,5,True,True
6,Nữ,6,True,True
7,Nữ,7,True,True
8,Nữ,8,True,True
9,Nữ,9,True,True


## Ví dụ 4: So sánh giữa các hàng

Ví dụ sau đây không sử dụng hàm `transform()`. Quan sát bộ số liệu dưới đây. Đây là số liệu của hai người, quan sát tại nhiều thời điểm nối tiếp nhau.

In [13]:
d4 = pd.DataFrame({
    "person": ["Person 1"] * 5 + ["Person 2"] * 5,
    "value": [1, 2, 5, 6, 9, 2, 4, 5, 8, 10]
})

d4

,person,value
0,Person 1,1
1,Person 1,2
2,Person 1,5
3,Person 1,6
4,Person 1,9
5,Person 2,2
6,Person 2,4
7,Person 2,5
8,Person 2,8
9,Person 2,10


Nếu bạn muốn so sánh giá trị của một hàng với hàng kế tiếp của họ, bạn sẽ cần tạo ra một cột để giữ giá trị này. Chúng ta có thể tạo ra cột đó bằng hàm `shift()`.

In [14]:
d4["value_shifted1"] = d4["value"].shift(1)
d4

,person,value,value_shifted1
0,Person 1,1,NaN
1,Person 1,2,1.0
2,Person 1,5,2.0
3,Person 1,6,5.0
4,Person 1,9,6.0
5,Person 2,2,9.0
6,Person 2,4,2.0
7,Person 2,5,4.0
8,Person 2,8,5.0
9,Person 2,10,8.0


Như bạn thấy, khi sử dụng hàm `shift()` và cung cấp đối số với giá trị là 1, chúng ta đang đẩy các giá trị trong cột `"value"` xuống một hàng. Giá trị đầu tiên sẽ là NA. Nếu cần bạn có thể thay giá trị này bằng một giá trị khác phù hợp (ví dụ, 0).

Hàm `shift()` cũng hoạt động với `groupby()`.

In [15]:
d4["value_shifted2"] = d4.groupby("person")["value"].shift(1).fillna(0, downcast="infer")
d4

,person,value,value_shifted1,value_shifted2
0,Person 1,1,NaN,0
1,Person 1,2,1.0,1
2,Person 1,5,2.0,2
3,Person 1,6,5.0,5
4,Person 1,9,6.0,6
5,Person 2,2,9.0,0
6,Person 2,4,2.0,2
7,Person 2,5,4.0,4
8,Person 2,8,5.0,5
9,Person 2,10,8.0,8


Và như vậy bạn có thể tính được độ "tăng" của các giá trị sau mỗi thời điểm.

In [16]:
d4["value_change"] = d4["value"] - d4["value_shifted2"]
d4

,person,value,value_shifted1,value_shifted2,value_change
0,Person 1,1,NaN,0,1
1,Person 1,2,1.0,1,1
2,Person 1,5,2.0,2,3
3,Person 1,6,5.0,5,1
4,Person 1,9,6.0,6,3
5,Person 2,2,9.0,0,2
6,Person 2,4,2.0,2,2
7,Person 2,5,4.0,4,1
8,Person 2,8,5.0,5,3
9,Person 2,10,8.0,8,2


## Mở rộng về `groupby()`

Bên cạnh nhóm theo nội dung trong một cột có sẵn, bạn cũng có thể sử dụng hàm lambda. Xem bộ số liệu sau.

In [17]:
d5 = pd.DataFrame({
    "value1": range(10),
    "value2": range(5, 15)
})
d5

,value1,value2
0,0,5
1,1,6
2,2,7
3,3,8
4,4,9
5,5,10
6,6,11
7,7,12
8,8,13
9,9,14


Vì một lí do gì đó, bạn muốn nhóm theo các hàng chẵn và lẽ, rồi tính trung bình của các giá trị. Bình thường bạn sẽ làm như sau.

In [18]:
d5["group_2"] = d5.index % 2
d5

,value1,value2,group_2
0,0,5,0
1,1,6,1
2,2,7,0
3,3,8,1
4,4,9,0
5,5,10,1
6,6,11,0
7,7,12,1
8,8,13,0
9,9,14,1


In [19]:
d5.groupby("group_2").mean()

,value1,value2
group_2,,
0,4.0,9.0
1,5.0,10.0


Bạn có thể làm đơn giản hơn mà không cần tạo nhóm mới; hãy sử dụng hàm lambda trong `groupby()`. Hàm này sẽ chạy trên index của data frame.

In [20]:
d5.drop(columns=["group_2"], inplace=True)
d5.groupby(lambda x: x % 2).mean()

,value1,value2
0,4.0,9.0
1,5.0,10.0


Bạn cũng có thể `transform()` số liệu nếu cần.

In [21]:
d5.groupby(lambda x: x % 2).transform("mean")

,value1,value2
0,4.0,9.0
1,5.0,10.0
2,4.0,9.0
3,5.0,10.0
4,4.0,9.0
5,5.0,10.0
6,4.0,9.0
7,5.0,10.0
8,4.0,9.0
9,5.0,10.0


---

[Bài trước](./08_agg.ipynb) - [Danh sách bài](../README.md) - [Bài sau]()